# Install `simex`

In [1]:
%%bash
#clone repo of `sipecam-metadata-extactor` in server
#git clone git@github.com:CONABIO/sipecam-metadata-extractor.git ~/
cd ../src
#install in editable mode
pip install -q -e .

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


# List of files to extract metadata

In [37]:
%%bash
dir_with_sipecam_data=/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/
list_of_files_and_subdirectories_to_extract_metadata --input_directory $dir_with_sipecam_data

## Check

In [38]:
%%bash
ls -lh /shared_volume/sipecam_files_to_extract_metadata_from_*

-rw-r--r-- 1 root root 3.2M Dec 15 10:11 /shared_volume/sipecam_files_to_extract_metadata_from_15-12-2021.txt


In [39]:
%%bash
ls -lh /shared_volume/sipecam_subdirectories_*

-rw-r--r-- 1 root root 1.8K Dec 15 10:11 /shared_volume/sipecam_subdirectories_15-12-2021.txt


In [40]:
%%bash
wc -l /shared_volume/sipecam_files_to_extract_metadata_from_*

29079 /shared_volume/sipecam_files_to_extract_metadata_from_15-12-2021.txt


In [41]:
%%bash
wc -l /shared_volume/sipecam_subdirectories_*

20 /shared_volume/sipecam_subdirectories_15-12-2021.txt


In [42]:
%%bash
#check every first subdirectory has a JPG, WAV or AVI file
SAVEIFS=$IFS;IFS=$(echo -en "\n\b")
for f in $(cat /shared_volume/sipecam_subdirectories_*.txt)
  do echo "$f"
     ls "$f"|grep "[WAV|wav|JPG|jpg|AVI|avi]$"|head -n 1
done
IFS=$SAVEIFS

/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico
20210728_180000.WAV
/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Audible
20210728_093000.WAV
/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_2/Ultrasonico
20210728_180000.WAV
/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_2/Audible
20210728_105000.WAV
/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_3/Nueva carpeta/Unidad de USB
20210728_141000.WAV
/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia 1339/1339_2/Audio/Ultrasonico
20210729_102000.WAV
/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia 1339/1339_2/Audio/Audible
20210729_180000.WAV
/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia 1339/1339_4/Audible
20210729_133000.WAV
/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia 1339/1339_1/Audio

# Extract metadata and ingest it

In [44]:
%%bash
file_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0049.JPG"
extract_metadata_and_ingest_it --input_file "$file_to_be_processed"

In [45]:
%%bash
file_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0012.AVI"
extract_metadata_and_ingest_it --input_file "$file_to_be_processed"

In [46]:
%%bash
file_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico/20210806_044000.WAV"
extract_metadata_and_ingest_it --input_file "$file_to_be_processed"

## Check

In [47]:
%%bash
file_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0049.JPG"
dirname_file_to_be_processed=$(dirname "$file_to_be_processed")
dir_for_logs=$(find "$dirname_file_to_be_processed" -name 'logs*')
filename_to_be_processed=$(basename "$file_to_be_processed")
cat "$dir_for_logs/$filename_to_be_processed.logs"

2021-12-15 10:12:49,631 [INFO]  extraction of metadata and ingestion of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0049.JPG
2021-12-15 10:12:49,632 [INFO]  Read metadata of JPG
2021-12-15 10:12:49,632 [INFO]  {}
2021-12-15 10:12:49,633 [INFO]  Getting node nomenclature and cumulus name from zendro
2021-12-15 10:12:49,633 [INFO]  Copying file to 3/1_3_1_28/2021-10-01/JPG


In [48]:
%%bash
file_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0012.AVI"
dirname_file_to_be_processed=$(dirname "$file_to_be_processed")
dir_for_logs=$(find "$dirname_file_to_be_processed" -name 'logs*')
filename_to_be_processed=$(basename "$file_to_be_processed")
cat "$dir_for_logs/$filename_to_be_processed.logs"

2021-12-15 10:12:52,320 [INFO]  extraction of metadata and ingestion of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0012.AVI
2021-12-15 10:12:52,321 [INFO]  Read metadata of AVI
2021-12-15 10:12:52,321 [INFO]  {}
2021-12-15 10:12:52,322 [INFO]  Getting node nomenclature and cumulus name from zendro
2021-12-15 10:12:52,322 [INFO]  Copying file to 3/1_3_1_28/2021-10-01/AVI


In [49]:
%%bash
file_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico/20210806_044000.WAV"
dirname_file_to_be_processed=$(dirname "$file_to_be_processed")
dir_for_logs=$(find "$dirname_file_to_be_processed" -name 'logs*')
filename_to_be_processed=$(basename "$file_to_be_processed")
cat "$dir_for_logs/$filename_to_be_processed.logs"

2021-12-15 10:12:52,614 [INFO]  extraction of metadata and ingestion of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico/20210806_044000.WAV
2021-12-15 10:12:52,616 [INFO]  Read metadata of WAV
2021-12-15 10:12:52,616 [INFO]  {}
2021-12-15 10:12:52,617 [INFO]  Getting node nomenclature and cumulus name from zendro
2021-12-15 10:12:52,617 [INFO]  Copying file to 3/1_3_1_28/2021-10-01/WAV
